In [ ]:
import os
from pathlib import Path
import re
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.crs import CRS

In [ ]:
input_dir = Path("path/to/input_folder")
output_dir = Path("path/to/output_folder")
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Target CRS: SAD69 UTM zone 22S (EPSG:29182)
target_crs = CRS.from_epsg(29182)
background = 0
foreground = 1

def recode_for_mspa(data):
    # MSPA expects: 1=background, 2=foreground, 0=other
    return np.where(data == foreground, 2,
           np.where(data == background, 1, 0)).astype(np.uint8)

In [ ]:
for tif_file in input_dir.glob("*.tif"):
    # Extract year from filename (e.g., 'mosaic_1990.tif' -> '1990')
    match = re.search(r"mosaic_(\d{4})", tif_file.stem)
    if match:
        year = match.group(1)
        output_name = f"{year}_P_recoded.tif"
    else:
        output_name = f"{tif_file.stem}_P_recoded.tif"

    with rasterio.open(tif_file) as src:
        data = src.read(1)
        recoded = recode_for_mspa(data)
        transform, width, height = rasterio.warp.calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)
        profile = src.profile.copy()
        profile.update({
            'driver': 'GTiff',
            'dtype': 'uint8',
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height,
            'count': 1,
            'compress': 'lzw'
        })
        reprojected = np.empty((height, width), dtype=np.uint8)
        rasterio.warp.reproject(
            source=recoded,
            destination=reprojected,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=target_crs,
            resampling=Resampling.bilinear
        )
        output_path = output_dir / output_name
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(reprojected, 1)
